#  Scikit-Learn - Utilizando um modelo treinado

### Objetivo

> O objetivo principal é a elaboração de um código para **usar um modelo de Machine Learning já treinado para preencher dados faltantes em um novo dataset**. 
>
> Vamos supor que temos um conjunto de dados com informações de alunos (como idade, renda, cidade, nota etc.) e queremos **prever uma categoria** (por exemplo: se o aluno foi aprovado ou reprovado, ou qual tipo de bolsa ele pode receber) com base nas outras informações.
>

## 📌 Estrutura Geral do Código

1. **Carregar o modelo salvo**
2. **Ler o novo dataset**
3. **Separar colunas por tipo (numéricas e categóricas)**
4. **Preencher valores faltantes (imputação)**
5. **Transformar variáveis categóricas em números (One-Hot Encoding)**
6. **Padronizar os dados numéricos**
7. **Usar o modelo para fazer previsões**
8. **Adicionar as previsões no dataset original**
9. **Mostrar o resultado final**


### 🐍 Código - Carregando as bibliotecas

In [34]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from joblib import load

### 🔖 Explicações

| Módulo | O que faz |
|-------|-----------|
| `OneHotEncoder` | Transforma texto em números. Converte `"cidade"` em colunas como `"cidade_Sao_Paulo"`, `"cidade_Rio"` etc. |
| `StandardScaler` | Deixa números na mesma escala. Alguns modelos aprendem melhor quando os números estão equilibrados. |
| `SimpleImputer` | Preenche campos vazios. Substitui por média, mediana ou outro valor fixo. |
| `ColumnTransformer` | Aplica transformações específicas por tipo de coluna. Permite aplicar `StandardScaler` nas colunas numéricas e `OneHotEncoder` nas categóricas |
| `joblib.load` | Carrega um modelo já treinado |

### 🐍 Código - Carregar o modelo salvo

In [28]:
modelo = load('modelo-treinado/modelo_regressao_logistica.joblib')

### 🔖 Explicações

- Aqui estamos carregando um **modelo de Machine Learning já treinado**, que foi salvo anteriormente com o nome `modelo_regressao_logistica.joblib`.
- A função `load()` do módulo `joblib` serve para carregar modelos que foram salvos com `dump()` depois do treinamento.
- Esse modelo vai ser usado para **fazer previsões** em novos dados.

### 🐍 Código - Ler o novo dataset

In [8]:
df_novo = pd.read_csv('dataset/dataset_sudeste_simples_realista_outliers-1k.csv')
df_novo.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         1000 non-null   int64  
 1   idade      981 non-null    float64
 2   renda      986 non-null    float64
 3   cidade     993 non-null    object 
 4   categoria  990 non-null    object 
 5   nota       990 non-null    float64
 6   feedback   985 non-null    object 
dtypes: float64(3), int64(1), object(3)
memory usage: 54.8+ KB


### 🔖 Explicações

- Estamos lendo um arquivo `.csv` chamado `dataset_sudeste_simples_realista_outliers-20250618_112332.csv` usando o Pandas (`pd.read_csv`).
- Esse é o **conjunto de dados onde falta alguma informação** que queremos prever.
- Por exemplo, talvez a coluna `"categoria"` esteja vazia ou ausente, e queremos usar o modelo para completá-la.

### 🐍 Código - Separar colunas por tipo

In [45]:
colunas_numericas = ['idade', 'renda', 'nota']
colunas_categoricas = ['cidade', 'feedback']

### 🔖 Explicações

- Separamos as colunas do dataset em dois grupos:
  - **Numéricas**: que têm valores como números (ex: idade, renda, nota)
  - **Categóricas**: que têm valores como textos ou categorias (ex: cidade, feedback)

Essa separação é importante porque **precisamos preparar os dados de formas diferentes** dependendo do tipo.

### 🐍 Código - Imputação de valores faltantes

In [46]:
imputer = SimpleImputer(strategy='mean')
df_novo[colunas_numericas] = imputer.fit_transform(df_novo[colunas_numericas])

### 🔖 Explicações

- Valores faltantes são aqueles que estão vazios ou com `NaN` (Not a Number).
- Usamos o `SimpleImputer` com estratégia `'mean'` para **substituir esses valores pela média** das colunas numéricas.
- Isso é necessário pois modelos de ML **não aceitam valores faltantes**.

### 🐍 Código

In [47]:
df_novo[['idade','renda','nota']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   idade   1000 non-null   float64
 1   renda   1000 non-null   float64
 2   nota    1000 non-null   float64
dtypes: float64(3)
memory usage: 23.6 KB


### 🔖 Explicações

...

### 🐍 Código

In [48]:
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
categorias_codificadas = encoder.fit_transform(df_novo[colunas_categoricas])

df_categorias = pd.DataFrame(categorias_codificadas, columns=encoder.get_feature_names_out(colunas_categoricas))
df_processado = pd.concat([df_novo.reset_index(drop=True), df_categorias], axis=1)
df_processado.drop(colunas_categoricas, axis=1, inplace=True)

### 🔖 Explicações

- Modelos de ML **não entendem texto diretamente**. Então precisamos transformar palavras em números.
- O `OneHotEncoder` cria **colunas binárias (0 ou 1)** para representar cada categoria.
  - Exemplo: Se a coluna `"cidade"` tem os valores `"São Paulo"`, `"Rio"`, `"Curitiba"`, ele cria três colunas: `cidade_São Paulo`, `cidade_Rio`, `cidade_Curitiba`.

> ⚠️ Importante: usamos `handle_unknown='ignore'` para evitar erros caso apareça uma nova cidade no novo dataset que não estava nos dados de treino.

### 🐍 Código

In [49]:
df_categorias.info()
df_categorias.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   cidade_Belo Horizonte  1000 non-null   float64
 1   cidade_Rio de Janeiro  1000 non-null   float64
 2   cidade_São Paulo       1000 non-null   float64
 3   cidade_Vitória         1000 non-null   float64
 4   cidade_nan             1000 non-null   float64
 5   feedback_Bom           1000 non-null   float64
 6   feedback_Regular       1000 non-null   float64
 7   feedback_Ruim          1000 non-null   float64
 8   feedback_nan           1000 non-null   float64
dtypes: float64(9)
memory usage: 70.4 KB


,cidade_Belo Horizonte,cidade_Rio de Janeiro,cidade_São Paulo,cidade_Vitória,cidade_nan,feedback_Bom,feedback_Regular,feedback_Ruim,feedback_nan
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


### 🔖 Explicações

...

### 🐍 Código

In [50]:
df_processado.info()
df_processado[colunas_numericas].head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     1000 non-null   int64  
 1   idade                  1000 non-null   float64
 2   renda                  1000 non-null   float64
 3   categoria              990 non-null    object 
 4   nota                   1000 non-null   float64
 5   cidade_Belo Horizonte  1000 non-null   float64
 6   cidade_Rio de Janeiro  1000 non-null   float64
 7   cidade_São Paulo       1000 non-null   float64
 8   cidade_Vitória         1000 non-null   float64
 9   cidade_nan             1000 non-null   float64
 10  feedback_Bom           1000 non-null   float64
 11  feedback_Regular       1000 non-null   float64
 12  feedback_Ruim          1000 non-null   float64
 13  feedback_nan           1000 non-null   float64
dtypes: float64(12), int64(1), object(1)
memory usage: 109.5+ 

,idade,renda,nota
0,36.000000,7038.010000,2.9
1,43.054027,9306.040000,5.1
2,35.000000,5148.470000,9.4
3,16.000000,12597.910000,1.8
4,41.000000,11057.240000,0.3
5,51.000000,6113.010000,1.1
6,69.000000,7170.179635,6.4
7,31.000000,4359.390000,1.8
8,41.000000,10422.770000,2.8
9,40.000000,9635.700000,9.9


### 🐍 Código

In [53]:
df_processado=df_processado.dropna()
df_processado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 990 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     990 non-null    int64  
 1   idade                  990 non-null    float64
 2   renda                  990 non-null    float64
 3   categoria              990 non-null    object 
 4   nota                   990 non-null    float64
 5   cidade_Belo Horizonte  990 non-null    float64
 6   cidade_Rio de Janeiro  990 non-null    float64
 7   cidade_São Paulo       990 non-null    float64
 8   cidade_Vitória         990 non-null    float64
 9   cidade_nan             990 non-null    float64
 10  feedback_Bom           990 non-null    float64
 11  feedback_Regular       990 non-null    float64
 12  feedback_Ruim          990 non-null    float64
 13  feedback_nan           990 non-null    float64
dtypes: float64(12), int64(1), object(1)
memory usage: 116.0+ KB


### 🔖 Explicações

...

### 🐍 Código - Padronização dos dados numéricos

In [54]:
scaler = StandardScaler()
df_processado[colunas_numericas] = scaler.fit_transform(df_processado[colunas_numericas])

### 🔖 Explicações

- Muitos modelos de Machine Learning exigem que os dados numéricos estejam na mesma escala.
- O `StandardScaler` **padroniza os valores**, ou seja, transforma-os para ter média zero e desvio padrão igual a 1.
- Isso evita que colunas com números muito grandes (ex: renda em reais) dominem o modelo.

### 🐍 Código

In [55]:
df_processado[colunas_numericas].head(10)

,idade,renda,nota
0,-0.433375,-0.037364,-0.779476
1,0.001024,0.625957,-0.026122
2,-0.494957,-0.589989,1.446344
3,-1.665010,1.588715,-1.156153
4,-0.125467,1.138122,-1.669804
5,0.490351,-0.307895,-1.395857
6,1.598822,0.001291,0.419042
7,-0.741284,-0.820768,-1.156153
8,-0.125467,0.952562,-0.813719
9,-0.187048,0.722371,1.617561


### 🔖 Explicações

...

### 🐍 Código - Preparando os dados

In [57]:
# X = todas as colunas, exceto 'categoria'
X = df_processado.drop('categoria', axis=1)
X.info()

print(f"\n\n")

# y = a coluna que queremos prever
y = df_processado['categoria']
y.info()

<class 'pandas.core.frame.DataFrame'>
Index: 990 entries, 0 to 999
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     990 non-null    int64  
 1   idade                  990 non-null    float64
 2   renda                  990 non-null    float64
 3   nota                   990 non-null    float64
 4   cidade_Belo Horizonte  990 non-null    float64
 5   cidade_Rio de Janeiro  990 non-null    float64
 6   cidade_São Paulo       990 non-null    float64
 7   cidade_Vitória         990 non-null    float64
 8   cidade_nan             990 non-null    float64
 9   feedback_Bom           990 non-null    float64
 10  feedback_Regular       990 non-null    float64
 11  feedback_Ruim          990 non-null    float64
 12  feedback_nan           990 non-null    float64
dtypes: float64(12), int64(1)
memory usage: 108.3 KB



<class 'pandas.core.series.Series'>
Index: 990 entries, 0 to 999

### 🔖 Explicações

...

### 🐍 Código - Fazer previsões com o modelo carregado

In [58]:
previsoes = modelo.predict(X)

### 🔖 Explicações

...

### 🐍 Código

In [59]:
acuracia = accuracy_score(y, previsoes)
print(f'Acurácia do modelo: {acuracia:.2f}')

Acurácia do modelo: 0.46


### 🔖 Explicações

- **`accuracy_score()`**: é uma função do Scikit-learn que calcula a **acurácia**, ou seja, a porcentagem de previsões corretas.
- **`y`**: são os valores reais (corretos) que o modelo deveria prever.
- **`previsoes`**: são as previsões feitas pelo modelo.
- **Resultado:** um número entre 0 e 1 (ex: `0.85` = 85% de acerto)

Esse é um dos métodos mais simples e comuns para avaliar modelos de classificação.

### 🐍 Código

In [1]:
#

### 🔖 Explicações

...

### 🐍 Código

In [1]:
#

### 🔖 Explicações

...

### 🐍 Código

In [1]:
#

### 🔖 Explicações

...

### 🐍 Código

In [1]:
#

### 🔖 Explicações

...

### 🐍 Código

In [1]:
#

### 🔖 Explicações

...

### 🐍 Código

In [1]:
#

### 🔖 Explicações

...

### 🐍 Código

In [1]:
#

### 🔖 Explicações

...

### 🐍 Código

In [1]:
#

### 🔖 Explicações

...

### 🐍 Código

In [1]:
#

### 🔖 Explicações

...

### 🐍 Código

In [1]:
#

### 🔖 Explicações

...

### 🐍 Código

In [1]:
#

### 🔖 Explicações

...

### 🐍 Código

In [1]:
#

### 🔖 Explicações

...

### 🐍 Código

In [1]:
#

### 🔖 Explicações

...

### 🐍 Código

In [1]:
#

### 🔖 Explicações

...

### 🐍 Código

In [1]:
#

### 🔖 Explicações

...

### 🐍 Código

In [1]:
#

### 🔖 Explicações

...

### 🐍 Código

In [1]:
#

### 🔖 Explicações

...

### 🐍 Código

In [1]:
#

### 🔖 Explicações

...